In [1]:
from typing import Tuple
import pandas

from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json

from sklearn.metrics import mean_squared_error

/home/ankitapurv/.conda/envs/env-hackathon23_somnolence/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
df = pandas.read_excel(io = '../../data/processed/france.xlsx')

In [3]:
def make_dataset(df_processed: pandas.DataFrame, df_covid_measures: pandas.DataFrame = pandas.DataFrame()) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_mrd = df_processed[['Time', 'Unemployment_Rate_TOT']].rename(
        columns = {'Time': 'ds', 'Unemployment_Rate_TOT': 'y'}
    )
    df_mrd = df_mrd.drop(index = df_mrd[pandas.isnull(df_mrd['y'])].index, inplace = False)
    df_covid_holidays = pandas.DataFrame()
    return df_mrd, df_covid_holidays

In [4]:
df_mrd, _ = make_dataset(df)

In [5]:
def train_test_split(df_mrd: pandas.DataFrame, test_size: int = 12) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_test = df_mrd.tail(test_size)
    df_train = df_mrd.drop(index = df_mrd.tail(test_size).index, inplace = False)
    return df_train, df_test

In [6]:
df_train, df_test = train_test_split(df_mrd, 12)

In [7]:
model = Prophet().fit(df_train)

00:29:07 - cmdstanpy - INFO - Chain [1] start processing
00:29:07 - cmdstanpy - INFO - Chain [1] done processing


In [8]:
def test_model(df_test: pandas.DataFrame, model):
    "return predicted values and rmse"
    df_predicted = model.predict(df_test)
    rmse = mean_squared_error(y_true = df_test['y'], y_pred = df_predicted['yhat'], squared = True)
    return df_predicted, rmse

In [9]:
df_predicted, rmse = test_model(df_test, model)

In [10]:
rmse

0.06700005152690783

In [11]:
with open('france_prophet_benchmark_model.json', 'w') as f:
    f.write(model_to_json(model))

In [12]:
df_predicted

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-03-01,7.634662,7.043310,8.264055,7.634662,7.634662,0.021856,0.021856,0.021856,0.021856,0.021856,0.021856,0.0,0.0,0.0,7.656519
1,2022-04-01,7.605190,6.982226,8.219529,7.605190,7.605190,0.008025,0.008025,0.008025,0.008025,0.008025,0.008025,0.0,0.0,0.0,7.613215
2,2022-05-01,7.576667,6.983909,8.179187,7.576667,7.576667,0.001612,0.001612,0.001612,0.001612,0.001612,0.001612,0.0,0.0,0.0,7.578279
3,2022-06-01,7.547194,6.950116,8.189461,7.547194,7.548809,0.008744,0.008744,0.008744,0.008744,0.008744,0.008744,0.0,0.0,0.0,7.555938
4,2022-07-01,7.518672,6.938926,8.120815,7.516161,7.524982,0.039145,0.039145,0.039145,0.039145,0.039145,0.039145,0.0,0.0,0.0,7.557818
5,2022-08-01,7.489199,6.928056,8.094729,7.482015,7.501370,0.036454,0.036454,0.036454,0.036454,0.036454,0.036454,0.0,0.0,0.0,7.525653
6,2022-09-01,7.459726,6.906031,8.176233,7.445317,7.479365,0.052469,0.052469,0.052469,0.052469,0.052469,0.052469,0.0,0.0,0.0,7.512195
7,2022-10-01,7.431204,6.885738,8.101326,7.410213,7.457688,0.054914,0.054914,0.054914,0.054914,0.054914,0.054914,0.0,0.0,0.0,7.486118
8,2022-11-01,7.401731,6.915728,8.073569,7.372815,7.438452,0.076243,0.076243,0.076243,0.076243,0.076243,0.076243,0.0,0.0,0.0,7.477974
9,2022-12-01,7.373209,6.810290,8.038937,7.336210,7.416348,0.072225,0.072225,0.072225,0.072225,0.072225,0.072225,0.0,0.0,0.0,7.445434


In [15]:
df_future = pandas.DataFrame(data = {'ds': ['2023-03-01', '2023-04-01']})

In [17]:
df_future_prediction = model.predict(df_future)

In [18]:
df_future_prediction

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-03-01,7.287642,6.723418,7.957026,7.287642,7.287642,0.051844,0.051844,0.051844,0.051844,0.051844,0.051844,0.0,0.0,0.0,7.339486
1,2023-04-01,7.258169,6.683456,7.872347,7.258169,7.258169,0.009988,0.009988,0.009988,0.009988,0.009988,0.009988,0.0,0.0,0.0,7.268157
